### Imports

In [ ]:
import sys
sys.path.insert(1, "../gapfilling/scripts/")

In [2]:
# loading gap filling module
from Gapfilling import *
from modelseed_gapfilling import *
import random

In [3]:
import psamm

### New functions

In [4]:
def make_model_copies(model, n = 10):
    d = {}
    N = n + 1
    for i in range(1, N):
        d[str(model) + "_" + str(i)] = fake_copy(model)
    return d

In [5]:
def remove_random_reaction(reference, model_dict, n = 1):
    """
    model_dict: dictionary of samples
    n: number of reactions to be removed from each model
    """
    n_samples = len(model_dict)
    N = n * n_samples
    
    reactions = list(reference.reactions)
    Random = random.sample(reactions, N)
    random_reactions = [i.id for i in Random]
    
    print(random_reactions)
    
    if N == n_samples:
        for i in range(len(model_dict)):
            list(model_dict.values())[i].remove_reactions([random_reactions[i]])
    else:
        c = 0
        for i in range(len(model_dict)):
            for j in range(c, c + n):
                list(model_dict.values())[i].remove_reactions([random_reactions[j]])
                c += 1
    return model_dict

### Testing
#### Loading models

In [6]:
e_coli = read_sbml_model("../experiment/models/references/iML1515.xml")
l_lactis = read_sbml_model("../experiment/models/references/iNF517.xml")
p_putida = read_sbml_model("../experiment/models/references/iJN1463.xml")

Using license file /home/fco/gurobi.lic
Academic license - for non-commercial use only - expires 2021-08-10


#### Checking objective functions

In [7]:
print("OBJECTIVE FUNCTIONS:\n")
print("E. coli:\n\n" + str(e_coli.objective) + "\n")
print("L. lactis:\n\n" + str(l_lactis.objective) + "\n")
print("P. putida:\n\n" + str(p_putida.objective) + "\n")

OBJECTIVE FUNCTIONS:

E. coli:

Maximize
1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685

L. lactis:

Maximize
1.0*BIOMASS_LLA - 1.0*BIOMASS_LLA_reverse_0796e

P. putida:

Maximize
1.0*BIOMASS_KT2440_WT3 - 1.0*BIOMASS_KT2440_WT3_reverse_d86d5



All three models are meant to optimized biomass production. Let's change the objective function of one of them in order to have a more diverse dataset.

In [8]:
p_putida.reactions.ACS

Reaction identifier,ACS
Name,Acetyl-CoA synthetase
Memory address,0x07ff5a82163d0
Stoichiometry,ac_c + atp_c + coa_c --> accoa_c + amp_c + ppi_c Acetate + ATP C10H12N5O13P3 + Coenzyme A --> Acetyl-CoA + AMP C10H12N5O7P + Diphosphate
GPR,PP_4487 or PP_4702 or PP_3071
Lower bound,0.0
Upper bound,999999.0


In [9]:
p_putida.objective = "ACS"

In [10]:
# Checking the fluxes of each model
print("FLUX VALUES:\n")
print("E. coli:\n\n" + str(e_coli.optimize().objective_value) + "\n")
print("L. lactis:\n\n" + str(l_lactis.optimize().objective_value) + "\n")
print("P. putida:\n\n" + str(p_putida.optimize().objective_value) + "\n")

FLUX VALUES:

E. coli:

0.8769972144269736

L. lactis:

0.04263460544337326

P. putida:

129.57999999999998



### Test 1: removal of 1 reaction

In [68]:
# Creating 10 copies of each model

e_coli_samples = make_model_copies(e_coli)
l_lactis_samples = make_model_copies(l_lactis)
p_putida_samples = make_model_copies(p_putida)

Now, we will remove one random reaction of every copy. 

In [69]:
print(len(list(e_coli_samples.values())[5].reactions))
print(len(list(l_lactis_samples.values())[5].reactions))
print(len(list(p_putida_samples.values())[5].reactions))

2712
754
2927


In [70]:
# removing one reaction from every model

remove_random_reaction(e_coli, e_coli_samples)
remove_random_reaction(p_putida, p_putida_samples)
remove_random_reaction(l_lactis, l_lactis_samples)

['EX_alaala_e', 'G3PD5', 'TYRTA', 'DSBCGT', 'TMPPP', 'GSNtex', '23CUMPtex', 'EX_metsox_S__L_e', 'XYLUt2pp', 'EX_rfamp_e']
['MBCOAi', 'PHADPCP90', 'PSERT', 'RHA140tpp', 'MUCCYCI', 'AGPAT161', 'DHAD1_copy2', 'ECOAH25', 'EX_glyphe_e', 'MDH2']
['EX_2hxic__L_e', 'EX_pyr_e', 'UGLDDS1', 'DHNPA_1', 'AMMQT8_2', 'THRA', 'ASPCT', 'UAAGLS1_1', 'PTRCabc', 'ACHBS']


{'iNF517_1': <Model iNF517 at 0x7ff511efdcd0>,
 'iNF517_2': <Model iNF517 at 0x7ff3b1cb02b0>,
 'iNF517_3': <Model iNF517 at 0x7ff3b5f82640>,
 'iNF517_4': <Model iNF517 at 0x7ff404117040>,
 'iNF517_5': <Model iNF517 at 0x7ff3e200af70>,
 'iNF517_6': <Model iNF517 at 0x7ff407f7a070>,
 'iNF517_7': <Model iNF517 at 0x7ff3d69fbca0>,
 'iNF517_8': <Model iNF517 at 0x7ff3d69fbd00>,
 'iNF517_9': <Model iNF517 at 0x7ff3cfc36130>,
 'iNF517_10': <Model iNF517 at 0x7ff3b5254310>}

In [71]:
print(len(list(e_coli_samples.values())[5].reactions))
print(len(list(l_lactis_samples.values())[5].reactions))
print(len(list(p_putida_samples.values())[5].reactions))

2711
753
2926


#### E. coli

In [72]:
# checking functionality of the models
for i in range(len(e_coli_samples)):
    print(list(e_coli_samples.keys())[i])
    print(list(e_coli_samples.values())[i].optimize().objective_value)

iML1515_1
0.8769972144269665
iML1515_2
0.8769972144269692
iML1515_3
0.0
iML1515_4
0.8769972144269693
iML1515_5
0.0
iML1515_6
0.8769972144269685
iML1515_7
0.8769972144269668
iML1515_8
0.8769972144269672
iML1515_9
0.8769972144269698
iML1515_10
0.8769972144269701


In [73]:
%%timeit -n1 -r1
# COBRA
for x in e_coli_samples:
    print(x)
    with suppress_stdout():
        sol = 0
        try:
            sol = cobra.flux_analysis.gapfill(e_coli_samples[x], e_coli, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iML1515_1
[]
iML1515_2
[]
iML1515_3
[<Reaction TYRTA at 0x7ff452fc90a0>]
iML1515_4
[]
iML1515_5
iML1515_6
[]
iML1515_7
[]
iML1515_8
[]
iML1515_9
[]
iML1515_10
[]
2min 53s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [74]:
%%timeit -n1 -r1
# GALLANT
for x in e_coli_samples:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(e_coli_samples[x], [e_coli], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iML1515_1
{'iML1515': [('EX_alaala_e', 'Exchange reaction')]}
iML1515_2
{'iML1515': []}
iML1515_3
{'iML1515': [('TYRTA', ['b0928', 'b3770', 'b4054'])]}
iML1515_4
{'iML1515': []}
iML1515_5
{'iML1515': [('TMPPP', ['b3993'])]}
iML1515_6
{'iML1515': [('GSNtex', ['b1377', 'b0241', 'b0929', 'b2215'])]}
iML1515_7
{'iML1515': [('23CUMPtex', ['b1377', 'b0241', 'b0929', 'b2215'])]}
iML1515_8
{'iML1515': [('EX_metsox_S__L_e', 'Exchange reaction')]}
iML1515_9
{'iML1515': [('XYLUt2pp', ['b3578', 'b3577', 'b3579'])]}
iML1515_10
{'iML1515': [('EX_rfamp_e', 'Exchange reaction')]}
3min 40s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


#### L. lactis

In [75]:
for i in range(len(l_lactis_samples)):
    print(list(l_lactis_samples.keys())[i])
    print(list(l_lactis_samples.values())[i].optimize().objective_value)

iNF517_1
0.04263460544337327
iNF517_2
0.04263460544337326
iNF517_3
0.0
iNF517_4
0.0
iNF517_5
0.04263460544337326
iNF517_6
0.04263460544337326
iNF517_7
0.04263460544337326
iNF517_8
0.0
iNF517_9
0.04263460544337326
iNF517_10
0.04263460544337326


In [76]:
%%timeit -n1 -r1
# COBRA
for x in l_lactis_samples:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(l_lactis_samples[x], l_lactis, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iNF517_1
iNF517_2
iNF517_3
iNF517_4
iNF517_5
iNF517_6
iNF517_7
iNF517_8
iNF517_9
iNF517_10
51.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [77]:
%%timeit -n1 -r1
# GALLANT
for x in l_lactis_samples:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(l_lactis_samples[x], [l_lactis], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iNF517_1
{}
iNF517_2
{}
iNF517_3
{}
iNF517_4
{}
iNF517_5
{}
iNF517_6
{}
iNF517_7
{}
iNF517_8
{}
iNF517_9
{}
iNF517_10
{}
52.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


#### P. putida

In [78]:
for i in range(len(p_putida_samples)):
    print(list(p_putida_samples.keys())[i])
    print(list(p_putida_samples.values())[i].optimize().objective_value)

iJN1463_1
129.57999999999998
iJN1463_2
129.57999999999998
iJN1463_3
129.57999999999998
iJN1463_4
129.58
iJN1463_5
129.57999999999998
iJN1463_6
129.58000000000004
iJN1463_7
129.57999999999998
iJN1463_8
129.57999999999998
iJN1463_9
129.57999999999998
iJN1463_10
129.57999999999998


In [79]:
%%timeit -n1 -r1
# COBRA
for x in p_putida_samples:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(p_putida_samples[x], p_putida, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iJN1463_1
[]
iJN1463_2
[]
iJN1463_3
[]
iJN1463_4
[]
iJN1463_5
[]
iJN1463_6
[]
iJN1463_7
[]
iJN1463_8
[]
iJN1463_9
[]
iJN1463_10
[]
3min 20s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [80]:
%%timeit -n1 -r1
# GALLANT
for x in p_putida_samples:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(p_putida_samples[x], [p_putida], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iJN1463_1
{'iJN1463': []}
iJN1463_2
{'iJN1463': []}
iJN1463_3
{'iJN1463': []}
iJN1463_4
{'iJN1463': [('RHA140tpp', [])]}
iJN1463_5
{'iJN1463': []}
iJN1463_6
{'iJN1463': []}
iJN1463_7
{'iJN1463': []}
iJN1463_8
{'iJN1463': []}
iJN1463_9
{'iJN1463': [('EX_glyphe_e', 'Exchange reaction')]}
iJN1463_10
{'iJN1463': []}
3min 21s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Test 2: removal of 2 reactions

In [81]:
e_coli_samples = make_model_copies(e_coli)
l_lactis_samples = make_model_copies(l_lactis)
p_putida_samples = make_model_copies(p_putida)

In [82]:
remove_random_reaction(e_coli, e_coli_samples, n = 2)
remove_random_reaction(p_putida, p_putida_samples, n = 2)
remove_random_reaction(l_lactis, l_lactis_samples, n = 2)

['14GLUCANabcpp', 'FE2tex', 'EX_crn__D_e', 'GLYC3Ptex', 'ACtex', 'PGPP161pp', 'DURIK1', 'HISTD', 'GPDDA2', 'GART', 'HCYSMT', 'NODOx', 'NNATr', 'G3PD5', 'SARCOX', 'PPALtex', 'FE3DCITtonex', 'SBTPD', 'LPLIPAL2E161', 'PGI']
['EX_phedca_e', 'PHADPC140', 'PE141abcpp', 'RECOAH25', 'EX_aso4_e', 'EX_adocbl_e', 'DAAD7', 'EX_pptrn_e', 'ADOCBIK', 'ACOAD29f', 'ACSPHAC101', 'CITtex', 'KARA1', 'HMSH', 'NTD6', 'AMPTASEHG', 'EX_acpptrn_e', 'ACACtex', '3HAACOAT140', 'APG3PAT120']
['S6PG', 'DNAS_LLA', 'RNDR3', 'MANAO', 'PGCD', 'GMPS2', 'HSK', 'IGPDH_1', 'DADK', 'LYSTRS_1', 'ADEt2', 'GLYCt', 'EX_btd_RR_e', 'SUCpts', 'TDPDRR_1', 'UGT2_LLA', 'ACHBS', 'VALTRS_1', 'OCBT_1', 'araphe1']


{'iNF517_1': <Model iNF517 at 0x7ff37c946eb0>,
 'iNF517_2': <Model iNF517 at 0x7ff410ae00d0>,
 'iNF517_3': <Model iNF517 at 0x7ff3929ef7f0>,
 'iNF517_4': <Model iNF517 at 0x7ff404218e20>,
 'iNF517_5': <Model iNF517 at 0x7ff51b6d7190>,
 'iNF517_6': <Model iNF517 at 0x7ff37f339eb0>,
 'iNF517_7': <Model iNF517 at 0x7ff510e4e820>,
 'iNF517_8': <Model iNF517 at 0x7ff5121bcd60>,
 'iNF517_9': <Model iNF517 at 0x7ff50c2f3520>,
 'iNF517_10': <Model iNF517 at 0x7ff386fc3ee0>}

In [83]:
print(len(list(e_coli_samples.values())[5].reactions))
print(len(list(l_lactis_samples.values())[5].reactions))
print(len(list(p_putida_samples.values())[5].reactions))

2710
752
2925


#### E. coli

In [85]:
# checking functionality of the models
for i in range(len(e_coli_samples)):
    print(list(e_coli_samples.keys())[i])
    print(list(e_coli_samples.values())[i].optimize().objective_value)

iML1515_1
0.8769200461250599
iML1515_2
0.8769972144269671
iML1515_3
0.8769972144269651
iML1515_4
0.0
iML1515_5
0.8769972144269718
iML1515_6
0.8769972144269661
iML1515_7
0.0
iML1515_8
0.8769972144269708
iML1515_9
0.876997214426969
iML1515_10
0.8696956584514168


In [86]:
%%timeit -n1 -r1
# COBRA
for x in e_coli_samples:
    print(x)
    with suppress_stdout():
        sol = 0
        try:
            sol = cobra.flux_analysis.gapfill(e_coli_samples[x], e_coli, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iML1515_1
[]
iML1515_2
[]
iML1515_3
[]
iML1515_4
[<Reaction HISTD at 0x7ff381146a00>]
iML1515_5
[]
iML1515_6
[]
iML1515_7
iML1515_8
[]
iML1515_9
[]
iML1515_10
[]
2min 50s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [87]:
%%timeit -n1 -r1
# GALLANT
for x in e_coli_samples:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(e_coli_samples[x], [e_coli], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iML1515_1
{'iML1515': [('FE2tex', ['b1377', 'b0241', 'b0929', 'b2215']), ('14GLUCANabcpp', ['b4032', 'b4034', 'b4035', 'b4033'])]}
iML1515_2
{'iML1515': [('EX_crn__D_e', 'Exchange reaction'), ('GLYC3Ptex', ['b1377', 'b0241', 'b0929', 'b2215'])]}
iML1515_3
{'iML1515': [('ACtex', ['b1377', 'b0241', 'b0929', 'b2215'])]}
iML1515_4
{'iML1515': [('HISTD', ['b2020'])]}
iML1515_5
{'iML1515': []}
iML1515_6
{'iML1515': []}
iML1515_7
{'iML1515': [('NNATr', ['b0639'])]}
iML1515_8
{'iML1515': [('PPALtex', ['b1377', 'b0241', 'b0929', 'b2215'])]}
iML1515_9
{'iML1515': [('FE3DCITtonex', ['b3005', 'b4291', 'b1252', 'b3006'])]}
iML1515_10
{'iML1515': []}
3min 35s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


#### L. lactis

In [88]:
for i in range(len(l_lactis_samples)):
    print(list(l_lactis_samples.keys())[i])
    print(list(l_lactis_samples.values())[i].optimize().objective_value)

iNF517_1
0.0
iNF517_2
0.04263460544337327
iNF517_3
0.042634605443373254
iNF517_4
0.04263460544337326
iNF517_5
0.0
iNF517_6
0.04263460544337326
iNF517_7
0.04263460544337326
iNF517_8
0.0
iNF517_9
0.0
iNF517_10
None


In [89]:
%%timeit -n1 -r1
# COBRA
for x in l_lactis_samples:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(l_lactis_samples[x], l_lactis, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iNF517_1
iNF517_2
iNF517_3
iNF517_4
iNF517_5
iNF517_6
iNF517_7
iNF517_8
iNF517_9
iNF517_10
51.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [90]:
%%timeit -n1 -r1
# GALLANT
for x in l_lactis_samples:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(l_lactis_samples[x], [l_lactis], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iNF517_1
{}
iNF517_2
{}
iNF517_3
{}
iNF517_4
{}
iNF517_5
{}
iNF517_6
{}
iNF517_7
{}
iNF517_8
{}
iNF517_9
{}
iNF517_10
{}
48.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


#### P. putida

In [91]:
for i in range(len(p_putida_samples)):
    print(list(p_putida_samples.keys())[i])
    print(list(p_putida_samples.values())[i].optimize().objective_value)

iJN1463_1
129.57999999999998
iJN1463_2
129.58000000000104
iJN1463_3
129.57999999999998
iJN1463_4
129.57999999999998
iJN1463_5
129.57999999999998
iJN1463_6
129.57999999999998
iJN1463_7
129.58
iJN1463_8
129.58
iJN1463_9
129.57999999999998
iJN1463_10
129.58000000000004


In [92]:
%%timeit -n1 -r1
# COBRA
for x in p_putida_samples:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(p_putida_samples[x], p_putida, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iJN1463_1
[]
iJN1463_2
[]
iJN1463_3
[]
iJN1463_4
[]
iJN1463_5
[]
iJN1463_6
[]
iJN1463_7
[]
iJN1463_8
[]
iJN1463_9
[]
iJN1463_10
[]
3min 22s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [93]:
%%timeit -n1 -r1
# GALLANT
for x in p_putida_samples:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(p_putida_samples[x], [p_putida], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iJN1463_1
{'iJN1463': [('EX_phedca_e', 'Exchange reaction')]}
iJN1463_2
{'iJN1463': []}
iJN1463_3
{'iJN1463': [('EX_adocbl_e', 'Exchange reaction'), ('EX_aso4_e', 'Exchange reaction')]}
iJN1463_4
{'iJN1463': [('EX_pptrn_e', 'Exchange reaction')]}
iJN1463_5
{'iJN1463': []}
iJN1463_6
{'iJN1463': [('CITtex', ['PP_1419', 'PP_2058'])]}
iJN1463_7
{'iJN1463': []}
iJN1463_8
{'iJN1463': []}
iJN1463_9
{'iJN1463': [('EX_acpptrn_e', 'Exchange reaction'), ('ACACtex', [])]}
iJN1463_10
{'iJN1463': []}
3min 39s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Test 3: removal of 5 reactions

In [94]:
e_coli_samples = make_model_copies(e_coli)
l_lactis_samples = make_model_copies(l_lactis)
p_putida_samples = make_model_copies(p_putida)

In [95]:
remove_random_reaction(e_coli, e_coli_samples, n = 5)
remove_random_reaction(p_putida, p_putida_samples, n = 5)
remove_random_reaction(l_lactis, l_lactis_samples, n = 5)

['SULFACtex', 'NTPP8', '3NTD4pp', 'URDGLYCD', 'LPLIPAL1G161pp', 'PRAIi', 'CCGS', 'DHFR', 'SUCCt2_2pp', 'CMtex', 'BUTSO3tex', 'PPM2', 'EX_octa_e', 'ACGAL1Ptex', 'SFGTHi', 'CD2t3pp', 'LPLIPAL2ATG141', 'OCTDPS', 'PPK2', '3HBZCT', 'FECRMR2', 'EX_gam_e', 'G3PIabcpp', 'LACZ', 'S2FE2SS2', 'NADPPPS', 'PYK', 'CU1abcpp', 'GALabcpp', 'HISTP', 'DASYN141', 'DHPS2', 'GLUt4pp', 'LIPAabcpp', 'OHPHM', 'G3PStex', 'FA80ACPHi', '2AGPE180tipp', 'DAGK180', 'APH180', 'G3PAT141', 'ETHSO3tex', 'METabcpp', 'CS1tex', 'GP4GH', 'EX_dopa_e', 'EAR181y', 'PG120abcpp', 'MCITS', 'LPLIPAL2E180']
['EX_5aptn_e', 'GLXO3r', 'KAT28', 'PEAMNtex', 'PRAMPC_copy1', 'ECOAH15', 'CARBSARAH', 'AKGt2rpp', 'ALAt2pp', '4PCP', 'NTD9pp', 'DAGK181', 'MN2t3pp', 'ECTOabcpp', 'RECOAH3', 'OCOAT4', 'HACD2i', 'AGMtex', 'PAPSR', 'GLCP3', 'TMPPP', 'MMETtex', 'CLPNH181pp', 'DURAD2', 'ECOAH8', 'ISOAML', 'LPCOXc', 'EX_algac_MG_23_e', 'PPGPPDP', 'PALDP', 'LYSTRS', 'EX_ag_e', 'FA161ACPHi', 'ADSS', 'AOBUTDs', '34DHPHEtex', 'D3ONEtpp', 'ECOAH29', 'PHAP2

{'iNF517_1': <Model iNF517 at 0x7ff3de3a4f40>,
 'iNF517_2': <Model iNF517 at 0x7ff35a7c1c70>,
 'iNF517_3': <Model iNF517 at 0x7ff31f9575e0>,
 'iNF517_4': <Model iNF517 at 0x7ff322b39700>,
 'iNF517_5': <Model iNF517 at 0x7ff326404370>,
 'iNF517_6': <Model iNF517 at 0x7ff322a46730>,
 'iNF517_7': <Model iNF517 at 0x7ff3372a5cd0>,
 'iNF517_8': <Model iNF517 at 0x7ff3a34f2760>,
 'iNF517_9': <Model iNF517 at 0x7ff35f7629d0>,
 'iNF517_10': <Model iNF517 at 0x7ff54435cd60>}

In [96]:
print(len(list(e_coli_samples.values())[5].reactions))
print(len(list(l_lactis_samples.values())[5].reactions))
print(len(list(p_putida_samples.values())[5].reactions))

2707
749
2922


#### E. coli

In [97]:
for i in range(len(e_coli_samples)):
    print(list(e_coli_samples.keys())[i])
    print(list(e_coli_samples.values())[i].optimize().objective_value)

iML1515_1
0.8769972144269703
iML1515_2
0.0
iML1515_3
0.8769972144269664
iML1515_4
0.0
iML1515_5
0.8769972144269689
iML1515_6
0.0
iML1515_7
0.0
iML1515_8
0.8769972144269679
iML1515_9
0.8769972144269665
iML1515_10
0.8769972144269738


In [98]:
%%timeit -n1 -r1
# COBRA
for x in e_coli_samples:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(e_coli_samples[x], e_coli, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iML1515_1
[]
iML1515_2
[<Reaction DHFR at 0x7ff381a65f40>, <Reaction PRAIi at 0x7ff381a65fa0>]
iML1515_3
[]
iML1515_4
iML1515_5
[]
iML1515_6
[<Reaction HISTP at 0x7ff3cd9a3910>]
iML1515_7
iML1515_8
[]
iML1515_9
[]
iML1515_10
[]
2min 52s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [99]:
%%timeit -n1 -r1
# GALLANT
for x in e_coli_samples:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(e_coli_samples[x], [e_coli], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iML1515_1
{'iML1515': [('SULFACtex', ['b1377', 'b0241', 'b0929', 'b2215'])]}
iML1515_2
{'iML1515': [('SUCCt2_2pp', ['b3528', 'b0010']), ('CMtex', ['b1377', 'b0241', 'b0929', 'b2215']), ('DHFR', ['b1606', 'b0048']), ('PRAIi', ['b1262'])]}
iML1515_3
{'iML1515': [('EX_octa_e', 'Exchange reaction'), ('BUTSO3tex', ['b1377', 'b0241', 'b0929', 'b2215']), ('ACGAL1Ptex', ['b1377', 'b0241', 'b0929', 'b2215'])]}
iML1515_4
{'iML1515': [('CD2t3pp', ['b0752', 'b3915']), ('OCTDPS', ['b3187'])]}
iML1515_5
{'iML1515': [('EX_gam_e', 'Exchange reaction')]}
iML1515_6
{'iML1515': [('HISTP', ['b2022'])]}
iML1515_7
{'iML1515': [('GLUt4pp', ['b3653']), ('DHPS2', ['b3177'])]}
iML1515_8
{'iML1515': [('G3PStex', ['b1377', 'b0241', 'b0929', 'b2215']), ('2AGPE180tipp', ['b2835'])]}
iML1515_9
{'iML1515': [('ETHSO3tex', ['b1377', 'b0241', 'b0929', 'b2215']), ('CS1tex', ['b1377', 'b0241', 'b0929', 'b2215'])]}
iML1515_10
{'iML1515': [('EX_dopa_e', 'Exchange reaction')]}
3min 48s ± 0 ns per loop (mean ± std. dev. of 1 

#### L. lactis

In [100]:
for i in range(len(l_lactis_samples)):
    print(list(l_lactis_samples.keys())[i])
    print(list(l_lactis_samples.values())[i].optimize().objective_value)

iNF517_1
0.04263460544337327
iNF517_2
0.0
iNF517_3
0.0
iNF517_4
0.04263460544337327
iNF517_5
0.0
iNF517_6
0.0
iNF517_7
0.04263460544337326
iNF517_8
0.0
iNF517_9
0.0
iNF517_10
0.0


In [101]:
%%timeit -n1 -r1
# COBRA
for x in l_lactis_samples:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(l_lactis_samples[x], l_lactis, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iNF517_1
iNF517_2
iNF517_3
iNF517_4
iNF517_5
iNF517_6
iNF517_7
iNF517_8
iNF517_9
iNF517_10
47.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [102]:
%%timeit -n1 -r1
# GALLANT
for x in l_lactis_samples:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(l_lactis_samples[x], [l_lactis], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iNF517_1
{}
iNF517_2
{}
iNF517_3
{}
iNF517_4
{}
iNF517_5
{}
iNF517_6
{}
iNF517_7
{}
iNF517_8
{}
iNF517_9
{}
iNF517_10
{}
50.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


#### P. putida

In [103]:
for i in range(len(p_putida_samples)):
    print(list(p_putida_samples.keys())[i])
    print(list(p_putida_samples.values())[i].optimize().objective_value)

iJN1463_1
129.57999999999998
iJN1463_2
129.58000000000004
iJN1463_3
129.57999999999998
iJN1463_4
129.57999999999998
iJN1463_5
129.57999999999998
iJN1463_6
129.58000000000104
iJN1463_7
129.57999999999998
iJN1463_8
129.57999999999998
iJN1463_9
129.57999999999998
iJN1463_10
129.57999999999998


In [104]:
%%timeit -n1 -r1
# COBRA
for x in p_putida_samples:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(p_putida_samples[x], p_putida, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iJN1463_1
[]
iJN1463_2
[]
iJN1463_3
[]
iJN1463_4
[]
iJN1463_5
[]
iJN1463_6
[]
iJN1463_7
[]
iJN1463_8
[]
iJN1463_9
[]
iJN1463_10
[]
3min 16s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [105]:
%%timeit -n1 -r1
# GALLANT
for x in p_putida_samples:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(p_putida_samples[x], [p_putida], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iJN1463_1
{'iJN1463': [('EX_5aptn_e', 'Exchange reaction'), ('PEAMNtex', [])]}
iJN1463_2
{'iJN1463': [('AKGt2rpp', ['PP_1400']), ('ALAt2pp', ['PP_4840'])]}
iJN1463_3
{'iJN1463': [('MN2t3pp', ['PP_4774'])]}
iJN1463_4
{'iJN1463': [('AGMtex', [])]}
iJN1463_5
{'iJN1463': [('MMETtex', [])]}
iJN1463_6
{'iJN1463': [('EX_algac_MG_23_e', 'Exchange reaction')]}
iJN1463_7
{'iJN1463': [('EX_ag_e', 'Exchange reaction')]}
iJN1463_8
{'iJN1463': [('EX_glu__L_e', 'Exchange reaction'), ('34DHPHEtex', []), ('D3ONEtpp', [])]}
iJN1463_9
{'iJN1463': [('AGMt2pp', ['PP_1355'])]}
iJN1463_10
{'iJN1463': [('PYRt2rpp', [])]}
3min 35s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Test 4: removal of 10 reactions

In [106]:
e_coli_samples = make_model_copies(e_coli)
l_lactis_samples = make_model_copies(l_lactis)
p_putida_samples = make_model_copies(p_putida)

In [107]:
remove_random_reaction(e_coli, e_coli_samples, n = 10)
remove_random_reaction(p_putida, p_putida_samples, n = 10)
remove_random_reaction(l_lactis, l_lactis_samples, n = 10)

['Ftex', 'EX_hxan_e', 'EX_glcr_e', 'SLNTabcpp', 'HISTP', 'UACGALPpp', 'DDPA', 'GLCP', 'EX_ade_e', 'ACOATA', 'LIPACabcpp', 'PPND', 'AKGt2rpp', 'FRULYStex', 'ACP1_FMN', 'GLNabcpp', 'POR5', 'EX_lac__L_e', 'SHCHD2', 'MDDCP4pp', 'NADPHXD', 'GALt2pp', 'DMSOtex', 'G3PD7', 'GLBRAN2', 'AGM4PCPpp', 'Htex', 'AGMHE', 'EX_malt_e', 'ACGALtex', '3OAR120', 'DAPabcpp', 'PLIPA1E160pp', '3OAR100', 'ACOAD2f', 'GLUt2rpp', 'EX_halipa_e', 'AMALT2', 'NTPP11', 'NH4tex', 'EX_hxa_e', 'UM4PL', 'EX_fuc__L_e', 'AGM3PApp', 'AST', 'ARBTNabcpp', 'NAt3_1p5pp', 'CPGNUtpp', 'SUCptspp', 'EX_cgly_e', 'GLUabcpp', 'EX_lys__L_e', 'EX_quin_e', 'HKNDDH', 'EX_2dglc_e', 'DHQTi', '3HPPtex', 'PETNT161pp', 'ADSS', 'GLYOX', 'EX_udpacgal_e', 'CMtex', 'PSD120', 'FEOXAMexs', 'SELtex', 'Kt2pp', 'AM3PA', '3AMPtex', 'ILEt3pp', 'IDOND', 'FECRMtonex', 'MANPGH', 'EX_duri_e', 'CTECOAI8', 'GALURtex', 'CYNTt2pp', 'APG3PAT181', 'CDGS', 'Kt3pp', 'PGM', 'G1SAT', 'DMPPS', 'PTRCtex', 'TAUDO', '2AGPA181tipp', 'E4PP', 'ALAALAr', 'SQtex', 'APG3PAT141', 

{'iNF517_1': <Model iNF517 at 0x7ff46937bb50>,
 'iNF517_2': <Model iNF517 at 0x7ff462201550>,
 'iNF517_3': <Model iNF517 at 0x7ff467cab460>,
 'iNF517_4': <Model iNF517 at 0x7ff4769af8e0>,
 'iNF517_5': <Model iNF517 at 0x7ff472fba4c0>,
 'iNF517_6': <Model iNF517 at 0x7ff467cab3d0>,
 'iNF517_7': <Model iNF517 at 0x7ff4d83106d0>,
 'iNF517_8': <Model iNF517 at 0x7ff4d6ba5580>,
 'iNF517_9': <Model iNF517 at 0x7ff4dfb5d370>,
 'iNF517_10': <Model iNF517 at 0x7ff488519fd0>}

In [108]:
print(len(list(e_coli_samples.values())[5].reactions))
print(len(list(l_lactis_samples.values())[5].reactions))
print(len(list(p_putida_samples.values())[5].reactions))

2702
744
2917


#### E. coli

In [109]:
for i in range(len(e_coli_samples)):
    print(list(e_coli_samples.keys())[i])
    print(list(e_coli_samples.values())[i].optimize().objective_value)

iML1515_1
0.0
iML1515_2
0.0
iML1515_3
0.8769972144269718
iML1515_4
0.0
iML1515_5
0.8769972144269693
iML1515_6
0.0
iML1515_7
0.8765145269377971
iML1515_8
0.816573060491076
iML1515_9
0.0
iML1515_10
0.8769972144269695


In [110]:
%%timeit -n1 -r1
# COBRA
for x in e_coli_samples:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(e_coli_samples[x], e_coli, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iML1515_1
[<Reaction DDPA at 0x7ff385f76d00>, <Reaction HISTP at 0x7ff386386a60>]
iML1515_2
iML1515_3
[]
iML1515_4
[<Reaction NH4tex at 0x7ff2f71b98b0>, <Reaction ACOAD2f at 0x7ff2f5334be0>]
iML1515_5
[]
iML1515_6
[<Reaction DHQTi at 0x7ff492c93f40>, <Reaction ADSS at 0x7ff5117c5e50>]
iML1515_7
[]
iML1515_8
[]
iML1515_9
iML1515_10
[]
2min 59s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [111]:
%%timeit -n1 -r1
# GALLANT
for x in e_coli_samples:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(e_coli_samples[x], [e_coli], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iML1515_1
{'iML1515': [('EX_hxan_e', 'Exchange reaction'), ('EX_ade_e', 'Exchange reaction'), ('EX_glcr_e', 'Exchange reaction'), ('Ftex', ['b1377', 'b0241', 'b0929', 'b2215']), ('DDPA', ['b0754', 'b2601', 'b1704']), ('HISTP', ['b2022'])]}
iML1515_2
{'iML1515': [('EX_lac__L_e', 'Exchange reaction'), ('AKGt2rpp', ['b2587']), ('FRULYStex', ['b1377', 'b0241', 'b0929', 'b2215']), ('SHCHD2', ['b3368']), ('PPND', ['b2600'])]}
iML1515_3
{'iML1515': [('EX_malt_e', 'Exchange reaction'), ('Htex', ['b1377', 'b0241', 'b0929', 'b2215']), ('GALt2pp', ['b2943']), ('DMSOtex', ['b1377', 'b0241', 'b0929', 'b2215']), ('ACGALtex', ['b1377', 'b0241', 'b0929', 'b2215'])]}
iML1515_4
{'iML1515': [('EX_halipa_e', 'Exchange reaction'), ('DAPabcpp', []), ('NH4tex', ['b1377', 'b0241', 'b0929', 'b2215']), ('GLUt2rpp', ['b4077']), ('3OAR100', ['b1093'])]}
iML1515_5
{'iML1515': [('EX_fuc__L_e', 'Exchange reaction'), ('EX_hxa_e', 'Exchange reaction'), ('EX_cgly_e', 'Exchange reaction'), ('CPGNUtpp', []), ('NAt3_1p5pp

#### L. lactis

In [112]:
for i in range(len(l_lactis_samples)):
    print(list(l_lactis_samples.keys())[i])
    print(list(l_lactis_samples.values())[i].optimize().objective_value)

iNF517_1
0.0
iNF517_2
0.0
iNF517_3
None
iNF517_4
None
iNF517_5
0.0
iNF517_6
0.0
iNF517_7
0.0
iNF517_8
0.0
iNF517_9
None
iNF517_10
0.0


In [113]:
%%timeit -n1 -r1
# COBRA
for x in l_lactis_samples:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(l_lactis_samples[x], l_lactis, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iNF517_1
iNF517_2
iNF517_3
iNF517_4
iNF517_5
iNF517_6
iNF517_7
iNF517_8
iNF517_9
iNF517_10
48.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [114]:
%%timeit -n1 -r1
# GALLANT
for x in l_lactis_samples:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(l_lactis_samples[x], [l_lactis], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iNF517_1
{}
iNF517_2
{}
iNF517_3
{}
iNF517_4
{}
iNF517_5
{}
iNF517_6
{}
iNF517_7
{}
iNF517_8
{}
iNF517_9
{}
iNF517_10
{}
50.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


#### P. putida

In [115]:
for i in range(len(p_putida_samples)):
    print(list(p_putida_samples.keys())[i])
    print(list(p_putida_samples.values())[i].optimize().objective_value)

iJN1463_1
129.57999999999998
iJN1463_2
129.57999999999998
iJN1463_3
129.57999999999998
iJN1463_4
129.58000000000004
iJN1463_5
129.57999999999998
iJN1463_6
129.57999999999998
iJN1463_7
129.57999999999998
iJN1463_8
129.58000000000004
iJN1463_9
129.57999999999998
iJN1463_10
129.58000000000004


In [116]:
%%timeit -n1 -r1
# COBRA
for x in p_putida_samples:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(p_putida_samples[x], p_putida, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iJN1463_1
[]
iJN1463_2
[]
iJN1463_3
[]
iJN1463_4
[]
iJN1463_5
[]
iJN1463_6
[]
iJN1463_7
[]
iJN1463_8
[]
iJN1463_9
[]
iJN1463_10
[]
3min 19s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [117]:
%%timeit -n1 -r1
# GALLANT
for x in p_putida_samples:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(p_putida_samples[x], [p_putida], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iJN1463_1
{'iJN1463': [('EX_R_3hocta_e', 'Exchange reaction'), ('ALGSex11', ['PP_1284']), ('FUMtex', []), ('GLCNtex', ['PP_1173'])]}
iJN1463_2
{'iJN1463': [('EX_gudptn_e', 'Exchange reaction'), ('PACt3', ['PP_3272']), ('QUINtpp', ['PP_2553']), ('D3ONEtpp', [])]}
iJN1463_3
{'iJN1463': [('EX_indole_e', 'Exchange reaction'), ('APCtex', []), ('HOMt2pp', ['PP_2239'])]}
iJN1463_4
{'iJN1463': [('EX_2dhglcn_e', 'Exchange reaction'), ('EX_cys__D_e', 'Exchange reaction'), ('ALAGLUabcpp', ['PP_0879', 'PP_0881', 'PP_0880', 'PP_0878', 'PP_0884', 'PP_0882', 'PP_0885'])]}
iJN1463_5
{'iJN1463': []}
iJN1463_6
{'iJN1463': [('EX_alahis_e', 'Exchange reaction'), ('OCDCAtexi', ['PP_1689'])]}
iJN1463_7
{'iJN1463': [('HG2tex', []), ('BTNt2ipp', [])]}
iJN1463_8
{'iJN1463': [('EX_cu2_e', 'Exchange reaction'), ('EX_tag182d9d12_e', 'Exchange reaction'), ('TOLtex', ['pWW0_127']), ('BALAt2pp', ['PP_4840', 'PP_0284'])]}
iJN1463_9
{'iJN1463': [('EX_R_3hphpa_e', 'Exchange reaction'), ('ALGSex10', ['PP_1284']), ('ARBT